# Wstępne przetworzenie danych odnoszacych sie do ratingow userow oraz ich podzial do user_features

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
df_ratings = pd.read_csv("ratings.csv")

In [3]:
df_ratings["timestamp"] = pd.to_datetime(df_ratings["timestamp"], unit="s")

df_ratings["day_of_week"] = df_ratings["timestamp"].dt.dayofweek
# df_ratings["day"] = df_ratings["timestamp"].dt.day
# df_ratings["month"] = df_ratings["timestamp"].dt.month
# df_ratings["year"] = df_ratings["timestamp"].dt.year
#
# df_ratings.drop(columns=["timestamp"], inplace=True)

In [4]:
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32000204 entries, 0 to 32000203
Data columns (total 5 columns):
 #   Column       Dtype         
---  ------       -----         
 0   userId       int64         
 1   movieId      int64         
 2   rating       float64       
 3   timestamp    datetime64[ns]
 4   day_of_week  int32         
dtypes: datetime64[ns](1), float64(1), int32(1), int64(2)
memory usage: 1.1 GB


In [5]:
# Zapisujemy dodatkowo przeksztalcenie time_stamp
# df_ratings.to_csv("ratings_improved.csv", index=False)

# Przygotowujemy cechy odpowiednio dla train i test


In [6]:
train_users, test_users = train_test_split(df_ratings['userId'].unique(), test_size=0.2, random_state=42)
df_train_ratings = df_ratings[df_ratings['userId'].isin(train_users)]
df_test_ratings  = df_ratings[df_ratings['userId'].isin(test_users)]

In [8]:
df_movies = pd.read_csv(
    "Movies_final_ML.csv",
    usecols=["movieId", "genres"]
)

## Train

In [9]:
df_train_ratings["timestamp"] = pd.to_datetime(df_train_ratings["timestamp"], unit="s")

df_train_ratings["day_of_week"] = df_train_ratings["timestamp"].dt.dayofweek

df_train_ratings["is_weekend"] = df_train_ratings["day_of_week"].isin([5, 6]).astype(int)

C:\Users\matimathew\AppData\Local\Temp\ipykernel_4876\3254962209.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_ratings["timestamp"] = pd.to_datetime(df_train_ratings["timestamp"], unit="s")
C:\Users\matimathew\AppData\Local\Temp\ipykernel_4876\3254962209.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_ratings["day_of_week"] = df_train_ratings["timestamp"].dt.dayofweek
C:\Users\matimathew\AppData\Local\Temp\ipykernel_4876\3254962209.py:5: SettingWithCopyWarning: 
A value is trying

In [11]:
df_merged_Tr = df_train_ratings.merge(df_movies, on="movieId", how="left")

df_merged_Tr["genres"] = df_merged_Tr["genres"].fillna("")  # w razie pustych
df_merged_Tr["genres_list"] = df_merged_Tr["genres"].apply(lambda g: g.split(","))

In [12]:
# Grupujemy po userID
df_user_base_Tr = df_merged_Tr.groupby("userId").agg(
    num_rating = ("rating", "count"),
    avg_rating = ("rating", "mean"),
    weekend_count = ("is_weekend", "sum")
).reset_index()

# Weekend_watcher = 1 jeśli > 50% ocen w weekend
df_user_base_Tr["weekend_ratio"] = df_user_base_Tr["weekend_count"] / df_user_base_Tr["num_rating"]
df_user_base_Tr["weekend_watcher"] = (df_user_base_Tr["weekend_ratio"] > 0.5).astype(int)

df_user_base_Tr.drop(columns=["weekend_ratio", "weekend_count"], inplace=True)


In [13]:
bins = [0, 3, 4, 5]
labels = ["negative", "neutral", "positive"]
df_user_base_Tr["type_of_viewer"] = pd.cut(
    df_user_base_Tr["avg_rating"],
    bins=bins,
    labels=labels,
    include_lowest=True
)

group_counts_Tr = df_user_base_Tr["type_of_viewer"].value_counts()

In [14]:
df_exploded_Tr = df_merged_Tr.explode("genres_list")

#Pozwoli zapobiec stworzeniu 2 kolumn, np. genre_Action i genre_ Action
df_exploded_Tr["genres_list"] = df_exploded_Tr["genres_list"].str.strip()

df_exploded_Tr = df_exploded_Tr[df_exploded_Tr["genres_list"] != ""]

df_genre_user_Tr = df_exploded_Tr.groupby(["userId", "genres_list"])["rating"].mean().reset_index()
df_genre_user_Tr.rename(columns={"genres_list": "genre", "rating": "avg_rating_genre"}, inplace=True)

df_genre_pivot_Tr = df_genre_user_Tr.pivot(
    index="userId",
    columns="genre",
    values="avg_rating_genre"
).reset_index()

df_genre_pivot_Tr.columns = ["userId"] + ["genre_" + str(col) for col in df_genre_pivot_Tr.columns if col != "userId"]
df_genre_pivot_Tr.head()


,userId,genre_Action,genre_Adventure,genre_Animation,genre_Comedy,genre_Crime,genre_Documentary,genre_Drama,genre_Family,genre_Fantasy,genre_History,genre_Horror,genre_Musical,genre_Mystery,genre_Romance,genre_Science Fiction,genre_TV Movie,genre_Thriller,genre_War,genre_Western
0,1,4.133333,3.333333,NaN,3.339623,4.285714,3.0,3.610000,2.666667,2.500000,3.526316,NaN,4.00,4.272727,3.487805,4.000000,NaN,3.666667,3.555556,3.00
1,2,4.000000,4.142857,4.875000,4.130435,4.000000,NaN,4.615385,4.692308,4.111111,NaN,1.000,NaN,4.750000,4.823529,NaN,NaN,4.300000,3.000000,NaN
2,3,3.583333,3.605263,3.961538,3.186275,3.346154,NaN,4.105263,3.833333,3.000000,4.875000,1.500,4.25,3.875000,3.637931,3.441176,NaN,3.307692,4.000000,3.75
3,4,2.454545,3.000000,NaN,3.000000,2.166667,NaN,2.222222,NaN,3.000000,2.500000,2.375,NaN,2.500000,2.000000,2.750000,NaN,2.142857,2.500000,NaN
4,6,4.295455,4.107143,2.000000,2.875000,NaN,NaN,3.785714,NaN,4.500000,5.000000,5.000,NaN,5.000000,3.250000,4.083333,NaN,4.636364,5.000000,4.00


In [15]:
df_user_features_Tr = df_user_base_Tr.merge(df_genre_pivot_Tr, on="userId", how="left")

df_user_features_Tr.to_csv("'../data/user_features_train.csv", index=False)

In [ ]:
df_user_features_Tr.info()
df_user_features_Tr.head()

## Test

In [16]:
df_test_ratings["timestamp"] = pd.to_datetime(df_test_ratings["timestamp"], unit="s")

df_test_ratings["day_of_week"] = df_test_ratings["timestamp"].dt.dayofweek

df_test_ratings["is_weekend"] = df_test_ratings["day_of_week"].isin([5, 6]).astype(int)

C:\Users\matimathew\AppData\Local\Temp\ipykernel_4876\3683249150.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_ratings["timestamp"] = pd.to_datetime(df_test_ratings["timestamp"], unit="s")
C:\Users\matimathew\AppData\Local\Temp\ipykernel_4876\3683249150.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_ratings["day_of_week"] = df_test_ratings["timestamp"].dt.dayofweek
C:\Users\matimathew\AppData\Local\Temp\ipykernel_4876\3683249150.py:5: SettingWithCopyWarning: 
A value is trying to 

In [17]:
df_merged_Te = df_test_ratings.merge(df_movies, on="movieId", how="left")

df_merged_Te["genres"] = df_merged_Te["genres"].fillna("")  # w razie pustych
df_merged_Te["genres_list"] = df_merged_Te["genres"].apply(lambda g: g.split(","))


In [18]:
# Grupujemy po userID
df_user_base_Te = df_merged_Te.groupby("userId").agg(
    num_rating = ("rating", "count"),
    avg_rating = ("rating", "mean"),
    weekend_count = ("is_weekend", "sum")  
).reset_index()

# Weekend_watcher = 1 jeśli > 50% ocen w weekend
df_user_base_Te["weekend_ratio"] = df_user_base_Te["weekend_count"] / df_user_base_Te["num_rating"]
df_user_base_Te["weekend_watcher"] = (df_user_base_Te["weekend_ratio"] > 0.5).astype(int)

df_user_base_Te.drop(columns=["weekend_ratio", "weekend_count"], inplace=True)


In [19]:
bins = [0, 3, 4, 5]
labels = ["negative", "neutral", "positive"]
df_user_base_Te["type_of_viewer"] = pd.cut(
    df_user_base_Te["avg_rating"],
    bins=bins,
    labels=labels,
    include_lowest=True
)

group_counts_Te = df_user_base_Te["type_of_viewer"].value_counts()

In [20]:
df_exploded_Te = df_merged_Te.explode("genres_list")

#Pozwoli zapobiec stworzeniu 2 kolumn, np. genre_Action i genre_ Action
df_exploded_Te["genres_list"] = df_exploded_Te["genres_list"].str.strip()

df_exploded_Te = df_exploded_Te[df_exploded_Te["genres_list"] != ""]

df_genre_user_Te = df_exploded_Te.groupby(["userId", "genres_list"])["rating"].mean().reset_index()
df_genre_user_Te.rename(columns={"genres_list": "genre", "rating": "avg_rating_genre"}, inplace=True)

df_genre_pivot_Te = df_genre_user_Te.pivot(
    index="userId",
    columns="genre",
    values="avg_rating_genre"
).reset_index()

df_genre_pivot_Te.columns = ["userId"] + ["genre_" + str(col) for col in df_genre_pivot_Te.columns if col != "userId"]
df_genre_pivot_Te.head()


,userId,genre_Action,genre_Adventure,genre_Animation,genre_Comedy,genre_Crime,genre_Documentary,genre_Drama,genre_Family,genre_Fantasy,genre_History,genre_Horror,genre_Musical,genre_Mystery,genre_Romance,genre_Science Fiction,genre_TV Movie,genre_Thriller,genre_War,genre_Western
0,5,3.666667,3.333333,3.666667,3.000000,2.625000,NaN,3.285714,3.500,3.000,3.500000,2.0,NaN,3.333333,3.250000,3.200000,NaN,3.466667,4.000000,3.0
1,7,3.000000,3.250000,3.500000,3.571429,3.777778,5.0,4.105263,3.625,3.625,5.000000,NaN,NaN,NaN,3.454545,3.166667,NaN,3.636364,NaN,3.5
2,13,3.631579,3.222222,NaN,3.461538,3.285714,NaN,3.775000,NaN,3.500,4.333333,NaN,4.0,3.400000,3.684211,3.500000,NaN,3.535714,4.000000,2.0
3,23,4.250000,4.214286,3.000000,4.437500,4.423077,5.0,4.597561,3.000,4.750,4.750000,4.0,NaN,4.500000,4.000000,4.210526,NaN,4.433333,4.611111,NaN
4,32,3.384615,3.153846,3.600000,3.285714,2.500000,NaN,4.142857,3.500,3.125,5.000000,NaN,5.0,3.500000,4.000000,3.428571,NaN,3.666667,NaN,4.0


In [21]:
df_user_features_Te = df_user_base_Te.merge(df_genre_pivot_Te, on="userId", how="left")

df_user_features_Te.to_csv("'../data/user_features_test.csv", index=False)


In [22]:
df_user_features_Te.info()
df_user_features_Te.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40190 entries, 0 to 40189
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   userId                 40190 non-null  int64   
 1   num_rating             40190 non-null  int64   
 2   avg_rating             40190 non-null  float64 
 3   weekend_watcher        40190 non-null  int32   
 4   type_of_viewer         40190 non-null  category
 5   genre_Action           39941 non-null  float64 
 6   genre_Adventure        40015 non-null  float64 
 7   genre_Animation        34940 non-null  float64 
 8   genre_Comedy           40102 non-null  float64 
 9   genre_Crime            39761 non-null  float64 
 10  genre_Documentary      14747 non-null  float64 
 11  genre_Drama            40177 non-null  float64 
 12  genre_Family           37750 non-null  float64 
 13  genre_Fantasy          39280 non-null  float64 
 14  genre_History          34717 non-null 

,userId,num_rating,avg_rating,weekend_watcher,type_of_viewer,genre_Action,genre_Adventure,genre_Animation,genre_Comedy,genre_Crime,...,genre_History,genre_Horror,genre_Musical,genre_Mystery,genre_Romance,genre_Science Fiction,genre_TV Movie,genre_Thriller,genre_War,genre_Western
0,5,33,3.272727,0,neutral,3.666667,3.333333,3.666667,3.000000,2.625000,...,3.500000,2.0,NaN,3.333333,3.250000,3.200000,NaN,3.466667,4.000000,3.0
1,7,44,3.636364,1,neutral,3.000000,3.250000,3.500000,3.571429,3.777778,...,5.000000,NaN,NaN,NaN,3.454545,3.166667,NaN,3.636364,NaN,3.5
2,13,65,3.646154,0,neutral,3.631579,3.222222,NaN,3.461538,3.285714,...,4.333333,NaN,4.0,3.400000,3.684211,3.500000,NaN,3.535714,4.000000,2.0
3,23,63,4.452381,0,positive,4.250000,4.214286,3.000000,4.437500,4.423077,...,4.750000,4.0,NaN,4.500000,4.000000,4.210526,NaN,4.433333,4.611111,NaN
4,32,44,3.613636,0,neutral,3.384615,3.153846,3.600000,3.285714,2.500000,...,5.000000,NaN,5.0,3.500000,4.000000,3.428571,NaN,3.666667,NaN,4.0
